# 0. Configuração de Ambiente **padrão**

###Copiar e colar os arquivos salvos na nuvem para esse ambiente de trabalho

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%bash
# Copiando e colando os arquivos da análise do Google Drive para esse ambiente de trabalho.
  # A barra invertida (\) ignora espaços no diretório, e o ponto (.) traz para o ambiente de trabalho atual.
cp /content/drive/MyDrive/Colab\ Notebooks/einstein_colabs/optativa_bioinfo/atividade_complementar/Chr7-reference.fasta.gz .
cp /content/drive/MyDrive/Colab\ Notebooks/einstein_colabs/optativa_bioinfo/atividade_complementar/caso3-tumor-pulmonar_R1.fastq.gz .
cp /content/drive/MyDrive/Colab\ Notebooks/einstein_colabs/optativa_bioinfo/atividade_complementar/caso3-tumor-pulmonar_R2.fastq.gz .

In [ ]:
%%bash
wget --quiet https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh
bash Miniforge3-Linux-x86_64.sh -b -p /usr/local -u
mamba init
mamba config set auto_activate_base true
mamba shell init --shell bash --root-prefix=~/.local/share/mamba
rm Miniforge3-Linux-x86_64.sh

In [ ]:
%%bash
# Instalação das bibliotecas necessárias
mamba install -c bioconda -c conda-forge fastqc bwa gatk4 cutadapt samtools bedtools ensembl-vep=115 -y

In [ ]:
%%bash
# Testando instalação de um deles
vep --help

In [ ]:
%%bash
# Organizando as pastas
mkdir -p raw fastqc cutadapt bwa gatk mutect vep reference

In [ ]:
%%bash
# Mover arquivos para as suas respectivas pastas e descompactar fasta.gz
mv *.fastq.gz raw/
gunzip -c Chr7-reference.fasta.gz > reference/Chr7-reference.fasta
rm -rf Chr7-reference.fasta.gz sample_data/

# 1. Controle de Qualidade

In [ ]:
%%bash
# Gerar o relatório da qualidade dos reads de sequenciamento
fastqc -o fastqc/ raw/caso3-tumor-pulmonar_R1.fastq.gz raw/caso3-tumor-pulmonar_R2.fastq.gz

In [ ]:
%%bash
# Filtrar sequências muito pequenas e cortar as pontas
cutadapt \
  -u 5 -U 5 \
  -u -5 -U -5 \
  -m 90 \
  -o cutadapt/caso3-tumor-pulmonar-trimmed_R1.fastq.gz \
  -p cutadapt/caso3-tumor-pulmonar-trimmed_R2.fastq.gz \
  raw/caso3-tumor-pulmonar_R1.fastq.gz \
  raw/caso3-tumor-pulmonar_R2.fastq.gz

# 2. Mapeamento com a referência do genoma humano

In [ ]:
%%bash
# Indexar o genoma humano de referência para realizar o mapeamento e posterior remoção de contaminante
samtools faidx reference/Chr7-reference.fasta
bwa index reference/Chr7-reference.fasta

In [ ]:
%%bash
# Alinhamento contra o genoma humano
bwa mem reference/Chr7-reference.fasta \
  -R '@RG\tID:sample1\tSM:sample1\tPL:ILLUMINA\tLB:lib1\tPU:unit1' \
  cutadapt/caso3-tumor-pulmonar-trimmed_R1.fastq.gz \
  cutadapt/caso3-tumor-pulmonar-trimmed_R2.fastq.gz \
  | samtools sort -o bwa/caso3-tumor-pulmonar-mapped.bam

In [ ]:
%%bash
# Indexar o bam para gerar o bai e visualizar no IGV
samtools index bwa/caso3-tumor-pulmonar-mapped.bam

#3. Chamada de Variantes

In [ ]:
%%bash
# Indexar a referência e BAM para chamada de variantes com GATK
samtools dict reference/Chr7-reference.fasta > reference/Chr7-reference.dict
samtools index bwa/caso3-tumor-pulmonar-mapped.bam

In [ ]:
%%bash
# Chamada de variantes com mutect2
 gatk Mutect2 -R reference/Chr7-reference.fasta \
    -I bwa/caso3-tumor-pulmonar-mapped.bam \
    -O mutect/caso3-tumor-pulmonar-unfiltered.vcf

In [ ]:
%%bash
# Filtragem de variantes
gatk FilterMutectCalls \
    -R reference/Chr7-reference.fasta \
    -V mutect/caso3-tumor-pulmonar-unfiltered.vcf \
    -O mutect/caso3-tumor-pulmonar-filtered.vcf

# 4. Anotação de Variantes

In [ ]:
%%bash
# Script oficial baixa cache do genoma humano (grande volume de dados)
vep_install -a cf -s homo_sapiens -y GRCh38 --NO_UPDATE

In [ ]:
%%bash
# Index VCF files para anotação
bgzip mutect/caso3-tumor-pulmonar-filtered.vcf
tabix -f -p vcf mutect/caso3-tumor-pulmonar-filtered.vcf.gz


In [ ]:
%%bash
# Anotação de variantes a nível de proteína
vep -i mutect/caso3-tumor-pulmonar-filtered.vcf.gz \
    --output_file vep/caso3-tumor-pulmonar-filtered.vep.tsv \
    --tab --force_overwrite \
    --offline --cache --dir_cache ~/.vep \
    --assembly GRCh38 \
    --symbol --hgvs --canonical --af_1kg --max_af \
    --everything --pick\
    --stats_file vep/caso3-tumor-pulmonar-filtered.vep.html
